In [1]:
%cd ..
from relbench.datasets import get_dataset

/home/lingze/embedding_fusion


In [2]:
dataset = get_dataset(name="rel-trial", download = True)
db = dataset.get_db()

Loading Database object from /home/lingze/.cache/relbench/rel-trial/db...
Done in 7.86 seconds.


In [3]:
# homoGraph
from utils.builder import HomoGraph, make_homograph_from_db
homoGraph = make_homograph_from_db(db)

In [4]:
from utils.preprocess import infer_type_in_db, tokenize_database

/home/lingze/anaconda3/envs/deepdb/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
col_type_dict = infer_type_in_db(db, True)

[rule 0]: interventionsInferred intervention_id from numerical as categorical
[rule 0]: interventions_studiesInferred id from numerical as categorical
[rule 0]: interventions_studiesInferred nct_id from numerical as categorical
[rule 0]: interventions_studiesInferred intervention_id from numerical as categorical
[rule 0]: facilities_studiesInferred id from numerical as categorical
[rule 0]: facilities_studiesInferred nct_id from numerical as categorical
[rule 0]: facilities_studiesInferred facility_id from numerical as categorical
[rule 0]: sponsorsInferred sponsor_id from numerical as categorical
[rule 1]: sponsorsInferred agency_class from text_embedded as categorical
[rule 0]: eligibilitiesInferred id from numerical as categorical
[rule 0]: eligibilitiesInferred nct_id from numerical as categorical
[rule 0]: reported_event_totalsInferred id from numerical as categorical
[rule 0]: reported_event_totalsInferred nct_id from numerical as categorical
[rule 0]: designsInferred id from num

In [6]:
col_type_dict

{'interventions': {'intervention_id': <stype.categorical: 'categorical'>,
  'mesh_term': <stype.text_embedded: 'text_embedded'>},
 'interventions_studies': {'id': <stype.categorical: 'categorical'>,
  'nct_id': <stype.categorical: 'categorical'>,
  'intervention_id': <stype.categorical: 'categorical'>,
  'date': <stype.timestamp: 'timestamp'>},
 'facilities_studies': {'id': <stype.categorical: 'categorical'>,
  'nct_id': <stype.categorical: 'categorical'>,
  'facility_id': <stype.categorical: 'categorical'>,
  'date': <stype.timestamp: 'timestamp'>},
 'sponsors': {'sponsor_id': <stype.categorical: 'categorical'>,
  'name': <stype.text_embedded: 'text_embedded'>,
  'agency_class': <stype.categorical: 'categorical'>},
 'eligibilities': {'id': <stype.categorical: 'categorical'>,
  'nct_id': <stype.categorical: 'categorical'>,
  'sampling_method': <stype.categorical: 'categorical'>,
  'gender': <stype.categorical: 'categorical'>,
  'minimum_age': <stype.text_embedded: 'text_embedded'>,
  '

In [7]:
tk_db = tokenize_database(db, col_type_dict, './tmp_test', True)

----------------> Tokenizing interventions each column
-> Load tokenized data from ./tmp_test/interventions.npy
----------------> Tokenizing interventions_studies each column
-> Load tokenized data from ./tmp_test/interventions_studies.npy
----------------> Tokenizing facilities_studies each column
-> Load tokenized data from ./tmp_test/facilities_studies.npy
----------------> Tokenizing sponsors each column
-> Load tokenized data from ./tmp_test/sponsors.npy
----------------> Tokenizing eligibilities each column
-> Load tokenized data from ./tmp_test/eligibilities.npy
----------------> Tokenizing reported_event_totals each column
-> Load tokenized data from ./tmp_test/reported_event_totals.npy
----------------> Tokenizing designs each column
-> Load tokenized data from ./tmp_test/designs.npy
----------------> Tokenizing conditions_studies each column
-> Load tokenized data from ./tmp_test/conditions_studies.npy
----------------> Tokenizing drop_withdrawals each column
-> Load tokenize

In [8]:
from utils.document import generate_document_given_table

In [9]:
db.table_dict.keys()

dict_keys(['interventions', 'interventions_studies', 'facilities_studies', 'sponsors', 'eligibilities', 'reported_event_totals', 'designs', 'conditions_studies', 'drop_withdrawals', 'studies', 'outcome_analyses', 'sponsors_studies', 'outcomes', 'conditions', 'facilities'])

In [10]:
table_name = 'interventions'

In [11]:
docs = generate_document_given_table(homoGraph, tk_db, table_name, True)

KeyError: 'interventions-intervention_id-0'

In [12]:
import torch
from torch_cluster import random_walk

In [13]:
g = homoGraph

In [15]:
row = torch.LongTensor(g.row)
col = torch.LongTensor(g.col)

In [16]:
pkys = list(range(len(tk_db.table_dict[table_name])))
len(pkys)

3462

In [17]:
gids = g.dbindex.get_global_ids(table_name, pkys)
gids = torch.LongTensor(gids)

In [18]:
walk_length = 20
p = 10
q = 0.5
walks = []
round = 10
for _ in range(round):
    walk = random_walk(row, col, gids, walk_length, p, q)

    # walk -> [num_target_node, walk_length]
    walk = walk.unsqueeze(1)
    # walk -> [num_target_node, 1, walk_length]
    walks.append(walk)

In [19]:
walks = torch.concatenate(walks, dim=1)

In [22]:
set(walks[0][0].tolist())

{0,
 157417,
 158067,
 159505,
 1975817,
 2577146,
 3905916,
 3907408,
 4125444,
 4502463,
 4878703}

In [23]:
walks.shape

torch.Size([3462, 10, 21])

In [24]:
from functools import reduce
from tqdm import tqdm
docs = []
# convert the walks to the tokens
for idx in tqdm(range(walks.shape[0]), leave=False):
    node_bags = walks[idx].tolist() # [round, walk_length]
    # for each walks, remove repeated node to construct subgraph.
    # make sure in each round of sample, the node is unqiue.
    doc_node_ids = reduce(lambda a, b: list(set(a)) + list(set(b)), node_bags)
    # [node_ids]
    table_pks = g.dbindex.get_tuple_positions(doc_node_ids)
    # [(table_name, pky)]
    
    tuple_list = [tk_db.get_tuple_attributes_set for table_name, pky in table_pks]
    doc = reduce(lambda a, b: a + b, tuple_list)
    docs.append(doc)

KeyError: 'interventions-intervention_id-0'

In [26]:
a = table_pks[0]

In [27]:
tk_db.get_tuple_attributes_set(a[0], a[1])

['interventions-intervention_id-0', 'interventions-mesh_term-tepotinib']